<a href="https://colab.research.google.com/github/natalievolk/LearnAI/blob/main/LearnAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# pillow to load image
from PIL import Image
import cv2
import requests
from io import BytesIO
#import urllib2



In [5]:
# training data
url_train = 'https://raw.githubusercontent.com/natalievolk/LearnAI/main/book30-listing-train.csv'
data_train = pd.read_csv(url_train, index_col=False, usecols=[2,5], encoding = "ISO-8859-1")
data_train.columns = ['jpg_url', 'classification']


# testing data
url_test = 'https://raw.githubusercontent.com/natalievolk/LearnAI/main/book30-listing-test.csv'
data_test = pd.read_csv(url_test, index_col=False, usecols=[2,5], encoding = "ISO-8859-1")
data_test.columns = ['jpg_url', 'classification']

data_train.head()
data_test.head(10)

,jpg_url,classification
0,http://ecx.images-amazon.com/images/I/510l0qhi...,28
1,http://ecx.images-amazon.com/images/I/51p5aUY%...,1
2,http://ecx.images-amazon.com/images/I/51pum0eW...,21
3,http://ecx.images-amazon.com/images/I/51PuTPPC...,0
4,http://ecx.images-amazon.com/images/I/51uNnIlI...,15
5,http://ecx.images-amazon.com/images/I/51VIlqRy...,10
6,http://ecx.images-amazon.com/images/I/41n7iZq-...,23
7,http://ecx.images-amazon.com/images/I/41xLYt-v...,9
8,http://ecx.images-amazon.com/images/I/51KyLXcz...,4
9,http://ecx.images-amazon.com/images/I/51Jku59B...,25


In [11]:
# MAKING TRAINING SET

file = open("train.txt", 'w')
#Image Preprocessing
 
for index, image_name in enumerate(data_train['jpg_url']):
    if image_name[-4:] == ".jpg":
      file.write(image_name + " " + str(data_train['classification'][index]) + "\n")

file.close()


file = open("test.txt",'w')
 
for index, image_name in enumerate(data_test['jpg_url']):
    if image_name[-4:] == ".jpg":
      file.write(image_name + " " + str(data_test['classification'][index]) + "\n")
 
file.close()


In [12]:
class ImagesDataset(Dataset):

    def __init__(self, text_file):
        self.df = pd.read_csv(text_file, sep=' ')
        self.df.columns = ['jpg_url', 'classification']

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        response = requests.get(self.df['jpg_url'][idx])
        img = Image.open(BytesIO(response.content)).resize((400, 400))
        pix = np.array(img)
 
        return (pix, self.df['classification'][idx])

#dataset = ImagesDataset('train.txt')
#dataset[10]
#len(dataset)
